In [1]:
import requests, pandas as pd

from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import DoubleType

# Configure Spark
conf = (SparkConf()
         .setMaster("local")
         .setAppName("App_name")
         .set("spark.executor.memory", "1g"))

sc = SparkContext(conf = conf)
spark = SparkSession(sparkContext=sc)

In [4]:
url_base = 'https://raw.githubusercontent.com/JeffSackmann/tennis_slam_pointbypoint/master/{}-{}-points.csv'

tournys = ['usopen']
years = [i for i in range(2020, 2022)]

chunks = []
for year in years:
    for tourny in tournys:
        url = url_base.format(year, tourny)
        req = requests.get(url=url)
        
        if req.status_code == 200:
            chunk = spark.createDataFrame(pd.read_csv(url).astype(str))
            chunks.append(chunk)
        else:
            continue

spark_df = chunks[0].select('*')
for chunk in chunks[1:]:
    spark_df = spark_df.union(chunk)

spark_df.printSchema()

c:\Users\Daniel\Documents\venvs\spark_env\lib\site-packages\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
c:\Users\Daniel\Documents\venvs\spark_env\lib\site-packages\pyspark\sql\pandas\conversion.py:371: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


root
 |-- match_id: string (nullable = true)
 |-- ElapsedTime: string (nullable = true)
 |-- SetNo: string (nullable = true)
 |-- P1GamesWon: string (nullable = true)
 |-- P2GamesWon: string (nullable = true)
 |-- SetWinner: string (nullable = true)
 |-- GameNo: string (nullable = true)
 |-- GameWinner: string (nullable = true)
 |-- PointNumber: string (nullable = true)
 |-- PointWinner: string (nullable = true)
 |-- PointServer: string (nullable = true)
 |-- Speed_KMH: string (nullable = true)
 |-- Rally: string (nullable = true)
 |-- P1Score: string (nullable = true)
 |-- P2Score: string (nullable = true)
 |-- P1Momentum: string (nullable = true)
 |-- P2Momentum: string (nullable = true)
 |-- P1PointsWon: string (nullable = true)
 |-- P2PointsWon: string (nullable = true)
 |-- P1Ace: string (nullable = true)
 |-- P2Ace: string (nullable = true)
 |-- P1Winner: string (nullable = true)
 |-- P2Winner: string (nullable = true)
 |-- P1DoubleFault: string (nullable = true)
 |-- P2DoubleFau

In [5]:
uso = spark_df[
    (spark_df['match_id'].contains('usopen')) 
    & (F.substring(spark_df['match_id'], -4,4).cast(DoubleType()) < 2000)
    ]

In [6]:
uso.toPandas()

,match_id,ElapsedTime,SetNo,P1GamesWon,P2GamesWon,SetWinner,GameNo,GameWinner,PointNumber,PointWinner,...,P2TurningPoint,ServeNumber,WinnerType,WinnerShotType,P1DistanceRun,P2DistanceRun,RallyCount,ServeWidth,ServeDepth,ReturnDepth
0,2020-usopen-1101,0:00:00,1,0,0,0,1,0,0X,0,...,nan,0,0,0,0.0,0.0,0,nan,nan,nan
1,2020-usopen-1101,0:00:00,1,0,0,0,1,0,0Y,0,...,nan,0,0,0,0.0,0.0,0,nan,nan,nan
2,2020-usopen-1101,0:00:00,1,0,0,0,1,0,1,1,...,nan,1,S,0,0.852,1.533,1,C,NCTL,nan
3,2020-usopen-1101,0:00:16,1,0,0,0,1,0,2,2,...,nan,1,0,F,24.451,19.851,8,BW,NCTL,D
4,2020-usopen-1101,0:01:01,1,0,0,0,1,0,3,1,...,nan,2,0,0,21.147,27.784,7,W,NCTL,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57550,2021-usopen-1701,2:12:59,3,4,5,0,10,0,178,2,...,nan,2,0,0,61.985,53.025,15,BW,NCTL,ND
57551,2021-usopen-1701,2:14:00,3,4,5,0,10,0,179,1,...,nan,2,0,0,26.825,26.446,4,BC,NCTL,D
57552,2021-usopen-1701,2:14:44,3,4,5,0,10,0,180,2,...,nan,2,0,F,28.78,19.018,5,BW,NCTL,ND
57553,2021-usopen-1701,2:15:22,3,4,5,0,10,0,181,1,...,nan,0,0,0,16.374,15.421,0,BC,CTL,nan
